In [2]:
import tensorflow as tf
tf.__version__


'2.12.0'

In [3]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [4]:
subject = 'Santa Catarina - Consumo de Cimento (t)'
split_index = 6 #Referente aos 7 anos de input  

In [5]:
data = pd.read_csv('../Inputs Atuais/2014_01_model_input_SC.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data

,Ano,Santa Catarina - IDH,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Longevidade,Santa Catarina - IDH Educacao,Santa Catarina - Consumo de Cimento (t)
0,2014,0.805257,1.560820e+08,6.814799e+06,20.085277,1.277386e+08,1598.981415,5.676413e+08,2.779340e+09,1.954361e+10,...,21.963483,1015.872526,1152.319790,21.397543,17.331617,78.730613,1.252238,0.867205,0.775185,3246.749286
1,2015,0.807829,1.574146e+08,6.792567e+06,20.038056,1.276578e+08,1488.521029,5.179392e+08,2.561731e+09,1.327596e+10,...,21.813817,1015.950998,1142.708354,21.308544,17.138396,78.237612,1.540937,0.877280,0.781469,3017.084229
2,2016,0.807723,1.585019e+08,6.760156e+06,20.026681,1.277375e+08,1430.320858,4.524144e+08,2.516859e+09,1.011370e+10,...,22.023495,1016.049222,1167.145418,21.317634,16.960969,77.364404,1.763965,0.875578,0.782603,2776.217486
3,2017,0.807152,1.591962e+08,6.740058e+06,20.047554,1.279508e+08,1433.051517,4.213689e+08,2.497710e+09,8.456309e+09,...,21.801864,1016.030235,1163.903842,21.473728,17.031652,76.967201,1.872343,0.873176,0.782385,2891.489000
4,2018,0.806257,1.591649e+08,6.752121e+06,20.052302,1.279548e+08,1463.153616,4.166580e+08,2.797145e+09,7.937760e+09,...,20.118575,1016.068237,1174.869796,21.497038,16.865702,76.096583,1.957243,0.870101,0.782783,2976.485000
5,2019,0.805098,1.583719e+08,6.760054e+06,20.041570,1.278279e+08,1592.460639,4.462625e+08,3.484252e+09,1.091282e+10,...,19.718452,1016.071217,1182.070943,21.348885,16.633933,75.707499,1.942028,0.866292,0.783036,3258.027000
6,2020,0.803198,1.564806e+08,6.769444e+06,20.007927,1.274268e+08,1768.446223,5.489412e+08,4.114911e+09,1.625333e+10,...,19.476367,1015.995806,1174.204399,21.164884,16.499861,75.914112,1.808726,0.861773,0.782040,3537.032000
7,2021,0.803216,1.532651e+08,6.765169e+06,19.992150,1.268376e+08,1855.413058,6.638811e+08,4.421848e+09,1.933031e+10,...,18.986511,1016.010789,1173.553772,20.928697,16.284921,75.986174,1.724014,0.862116,0.781316,4089.333000
8,2022,0.804236,1.493834e+08,6.758745e+06,19.973552,1.261213e+08,1866.444211,7.010332e+08,4.533216e+09,1.997888e+10,...,19.420495,1016.054127,1175.959349,20.784163,16.147783,76.019381,1.711897,0.865309,0.780526,4255.808000


In [6]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Santa Catarina - IDH,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Santa Catarina - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Longevidade,Santa Catarina - IDH Educacao
0,0.805257,1.560820e+08,6.814799e+06,20.085277,1.277386e+08,1598.981415,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,5.511795,21.963483,1015.872526,1152.319790,21.397543,17.331617,78.730613,1.252238,0.867205,0.775185
1,0.807829,1.574146e+08,6.792567e+06,20.038056,1.276578e+08,1488.521029,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,5.792915,21.813817,1015.950998,1142.708354,21.308544,17.138396,78.237612,1.540937,0.877280,0.781469
2,0.807723,1.585019e+08,6.760156e+06,20.026681,1.277375e+08,1430.320858,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,5.798844,22.023495,1016.049222,1167.145418,21.317634,16.960969,77.364404,1.763965,0.875578,0.782603
3,0.807152,1.591962e+08,6.740058e+06,20.047554,1.279508e+08,1433.051517,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,5.526597,21.801864,1016.030235,1163.903842,21.473728,17.031652,76.967201,1.872343,0.873176,0.782385
4,0.806257,1.591649e+08,6.752121e+06,20.052302,1.279548e+08,1463.153616,4.166580e+08,2.797145e+09,7.937760e+09,4.467878e+10,...,5.417481,20.118575,1016.068237,1174.869796,21.497038,16.865702,76.096583,1.957243,0.870101,0.782783
5,0.805098,1.583719e+08,6.760054e+06,20.041570,1.278279e+08,1592.460639,4.462625e+08,3.484252e+09,1.091282e+10,4.379096e+10,...,5.342752,19.718452,1016.071217,1182.070943,21.348885,16.633933,75.707499,1.942028,0.866292,0.783036
6,0.803198,1.564806e+08,6.769444e+06,20.007927,1.274268e+08,1768.446223,5.489412e+08,4.114911e+09,1.625333e+10,4.320994e+10,...,5.254689,19.476367,1015.995806,1174.204399,21.164884,16.499861,75.914112,1.808726,0.861773,0.782040


In [7]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0    3017.084229
1    2776.217486
2    2891.489000
3    2976.485000
4    3258.027000
5    3537.032000
6    4089.333000
7    4255.808000
8            NaN
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [8]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Santa Catarina - IDH,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Santa Catarina - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Longevidade,Santa Catarina - IDH Educacao
0,-1.179767,-1.873878,1.752430,2.009814,-0.651159,1.404170,1.779974,0.019134,2.003355,-1.880579,...,-0.305779,0.765874,-1.859406,-0.872979,0.095467,1.554644,1.437642,-1.860646,-1.076206,-2.200278
1,1.162093,-0.649751,0.883571,-0.575916,-1.376005,-0.180154,0.870373,-0.621387,0.401153,0.111653,...,1.307928,0.607450,-0.774934,-1.601545,-1.121593,0.665667,0.979381,-0.715841,1.387848,0.081909
2,1.065194,0.349054,-0.383139,-1.198788,-0.661149,-1.014914,-0.328802,-0.753466,-0.407217,1.154682,...,1.341962,0.829399,0.582492,0.250833,-0.997283,-0.150641,0.167705,0.168554,0.971618,0.493602
3,0.545696,0.986830,-1.168592,-0.055819,1.251182,-0.975748,-0.896969,-0.809829,-0.830898,0.955301,...,-0.220813,0.594798,0.320099,0.005115,1.137296,0.174558,-0.201508,0.598316,0.384145,0.414445
4,-0.269304,0.958127,-0.697163,0.204200,1.287492,-0.543997,-0.983185,0.071542,-0.963455,0.100493,...,-0.847165,-1.186992,0.845279,0.836357,1.456048,-0.588950,-1.010776,0.934975,-0.367907,0.559213
5,-1.323912,0.229618,-0.387107,-0.383490,0.149639,1.310643,-0.441390,2.094006,-0.202939,-0.441549,...,-1.276132,-1.610529,0.886471,1.382218,-0.569934,-1.655278,-1.372443,0.874641,-1.299498,0.651108


In [9]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0    3017.084229
1    2776.217486
2    2891.489000
3    2976.485000
4    3258.027000
5    3537.032000
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [10]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Santa Catarina - IDH,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Santa Catarina - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Longevidade,Santa Catarina - IDH Educacao
6,-1.851289,-1.212767,-0.018643,-1.576849,-1.943166,2.016188,1.16956,2.035192,0.985175,-0.705937,...,-1.368185,-1.412171,-0.143901,0.697496,-1.859643,-1.59586,-0.998008,0.317671,-1.647513,0.266084


In [11]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

6    4089.333
Name: Santa Catarina - Consumo de Cimento (t), dtype: float64

In [43]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=1000, verbose=1)
    # Método estocástico e learning rate=0.01
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(74, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(296, activation='tanh'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(148, activation='tanh'),
        tf.keras.layers.Dense(36, activation='tanh'),
        tf.keras.layers.Dense(18, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=5000, 
                      batch_size=len(train_input), 
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [44]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(10)]
    print(random_seeds)

    best_val_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, 0, seed)

        current_val_loss = min(history.history['val_loss'])
        print(f"val_loss: {current_val_loss}")

        if current_val_loss < best_val_loss:
            best_val_loss = current_val_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [45]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3571069361, 778615001, 1322324392, 2867196699, 2783886610, 212387672, 1617440382, 894072946, 1480511042, 1408278902]


Step: 0 ___________________________________________
Epoch 1448: early stopping
val_loss: 98.47267150878906
winner_seed: 3571069361


Step: 1 ___________________________________________
Epoch 1581: early stopping
val_loss: 513.5269165039062


Step: 2 ___________________________________________
Epoch 2377: early stopping
val_loss: 7400.11572265625


Step: 3 ___________________________________________
Epoch 2400: early stopping
val_loss: 19972.189453125


Step: 4 ___________________________________________
Epoch 1251: early stopping
val_loss: 53.38343048095703
winner_seed: 2783886610


Step: 5 ___________________________________________
Epoch 1040: early stopping
val_loss: 12966.0703125


Step: 6 ___________________________________________
Epoch 2181: early stopping
val_loss: 12406.076171875


Step: 7 ___________________________________________
Epoch 1403: early stopping

In [46]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/5000
1/1 [==============================] - 0s 332ms/step - loss: 9526559.0000 - val_loss: 12041163.0000
Epoch 2/5000
1/1 [==============================] - 0s 17ms/step - loss: 9157944.0000 - val_loss: 7469809.5000
Epoch 3/5000
1/1 [==============================] - 0s 13ms/step - loss: 9811309.0000 - val_loss: 9277735.0000
Epoch 4/5000
1/1 [==============================] - 0s 16ms/step - loss: 7913980.0000 - val_loss: 8054766.5000
Epoch 5/5000
1/1 [==============================] - 0s 12ms/step - loss: 8305336.5000 - val_loss: 8234789.5000
Epoch 6/5000
1/1 [==============================] - 0s 13ms/step - loss: 7424313.5000 - val_loss: 7955327.5000
Epoch 7/5000
1/1 [==============================] - 0s 12ms/step - loss: 6610593.5000 - val_loss: 7085329.0000
Epoch 8/5000
1/1 [==============================] - 0s 12ms/step - loss: 6931212.0000 - val_loss: 6773512.0000
Epoch 9/5000
1/1 [==============================] - 0s 13ms/step - loss: 5831984.0000 - val_loss: 9653252.0000

Epoch 75/5000
1/1 [==============================] - 0s 14ms/step - loss: 565008.6875 - val_loss: 15487.9727
Epoch 76/5000
1/1 [==============================] - 0s 12ms/step - loss: 620907.5625 - val_loss: 16770.9453
Epoch 77/5000
1/1 [==============================] - 0s 13ms/step - loss: 522279.5312 - val_loss: 11473.8828
Epoch 78/5000
1/1 [==============================] - 0s 17ms/step - loss: 577136.9375 - val_loss: 11567.4336
Epoch 79/5000
1/1 [==============================] - 0s 13ms/step - loss: 325433.7188 - val_loss: 16642.1973
Epoch 80/5000
1/1 [==============================] - 0s 13ms/step - loss: 472752.0312 - val_loss: 9255.3701
Epoch 81/5000
1/1 [==============================] - 0s 13ms/step - loss: 518780.7812 - val_loss: 8006.1426
Epoch 82/5000
1/1 [==============================] - 0s 13ms/step - loss: 496774.5312 - val_loss: 6924.7915
Epoch 83/5000
1/1 [==============================] - 0s 14ms/step - loss: 417067.7812 - val_loss: 3562.8599
Epoch 84/5000
1/1 [====

1/1 [==============================] - 0s 12ms/step - loss: 24842.6035 - val_loss: 185827.9219
Epoch 151/5000
1/1 [==============================] - 0s 12ms/step - loss: 42225.4531 - val_loss: 187593.2500
Epoch 152/5000
1/1 [==============================] - 0s 12ms/step - loss: 37370.1133 - val_loss: 177065.2812
Epoch 153/5000
1/1 [==============================] - 0s 12ms/step - loss: 39260.8711 - val_loss: 174387.1875
Epoch 154/5000
1/1 [==============================] - 0s 11ms/step - loss: 61426.2344 - val_loss: 175333.1562
Epoch 155/5000
1/1 [==============================] - 0s 11ms/step - loss: 32402.2578 - val_loss: 166112.1719
Epoch 156/5000
1/1 [==============================] - 0s 11ms/step - loss: 82124.7812 - val_loss: 195857.6719
Epoch 157/5000
1/1 [==============================] - 0s 12ms/step - loss: 57854.5898 - val_loss: 185459.7656
Epoch 158/5000
1/1 [==============================] - 0s 12ms/step - loss: 33106.3516 - val_loss: 168709.9531
Epoch 159/5000
1/1 [=====

1/1 [==============================] - 0s 12ms/step - loss: 36496.4258 - val_loss: 124202.3828
Epoch 225/5000
1/1 [==============================] - 0s 12ms/step - loss: 25344.8594 - val_loss: 129278.5234
Epoch 226/5000
1/1 [==============================] - 0s 12ms/step - loss: 36192.3789 - val_loss: 126127.9844
Epoch 227/5000
1/1 [==============================] - 0s 12ms/step - loss: 35949.9102 - val_loss: 123790.0703
Epoch 228/5000
1/1 [==============================] - 0s 12ms/step - loss: 35739.3281 - val_loss: 121972.4922
Epoch 229/5000
1/1 [==============================] - 0s 12ms/step - loss: 35546.2109 - val_loss: 120492.3750
Epoch 230/5000
1/1 [==============================] - 0s 12ms/step - loss: 35364.4727 - val_loss: 119233.8125
Epoch 231/5000
1/1 [==============================] - 0s 12ms/step - loss: 35191.4609 - val_loss: 118123.6172
Epoch 232/5000
1/1 [==============================] - 0s 12ms/step - loss: 35025.9102 - val_loss: 117115.3438
Epoch 233/5000
1/1 [=====

Epoch 299/5000
1/1 [==============================] - 0s 12ms/step - loss: 31293.7441 - val_loss: 87614.5547
Epoch 300/5000
1/1 [==============================] - 0s 16ms/step - loss: 31282.9902 - val_loss: 87423.4453
Epoch 301/5000
1/1 [==============================] - 0s 12ms/step - loss: 31272.6660 - val_loss: 87236.4375
Epoch 302/5000
1/1 [==============================] - 0s 13ms/step - loss: 31262.7500 - val_loss: 87053.3750
Epoch 303/5000
1/1 [==============================] - 0s 12ms/step - loss: 31253.2324 - val_loss: 86874.1094
Epoch 304/5000
1/1 [==============================] - 0s 12ms/step - loss: 31244.0859 - val_loss: 86698.6172
Epoch 305/5000
1/1 [==============================] - 0s 12ms/step - loss: 31235.3027 - val_loss: 86526.7500
Epoch 306/5000
1/1 [==============================] - 0s 13ms/step - loss: 31226.8652 - val_loss: 86358.5000
Epoch 307/5000
1/1 [==============================] - 0s 13ms/step - loss: 31218.7578 - val_loss: 86193.8516
Epoch 308/5000
1/1 

...

1/1 [==============================] - 0s 12ms/step - loss: 31022.4746 - val_loss: 78579.4766
Epoch 750/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.4629 - val_loss: 78568.5312
Epoch 751/5000
1/1 [==============================] - 0s 13ms/step - loss: 31022.4590 - val_loss: 78559.9062
Epoch 752/5000
1/1 [==============================] - 0s 16ms/step - loss: 31022.4473 - val_loss: 78552.6562
Epoch 753/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.4434 - val_loss: 78546.3594
Epoch 754/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.4316 - val_loss: 78540.8828
Epoch 755/5000
1/1 [==============================] - 0s 13ms/step - loss: 31022.4277 - val_loss: 78535.6875
Epoch 756/5000
1/1 [==============================] - 0s 13ms/step - loss: 31022.4219 - val_loss: 78530.8906
Epoch 757/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.4160 - val_loss: 78526.3828
Epoch 758/5000
1/1 [==============

1/1 [==============================] - 0s 12ms/step - loss: 31022.3359 - val_loss: 78456.4766
Epoch 825/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3340 - val_loss: 78453.6016
Epoch 826/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3340 - val_loss: 78450.8672
Epoch 827/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3281 - val_loss: 78448.1328
Epoch 828/5000
1/1 [==============================] - 0s 13ms/step - loss: 31022.3262 - val_loss: 78445.5312
Epoch 829/5000
1/1 [==============================] - 0s 16ms/step - loss: 31022.3223 - val_loss: 78442.9375
Epoch 830/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3203 - val_loss: 78440.3359
Epoch 831/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3184 - val_loss: 78437.8750
Epoch 832/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3184 - val_loss: 78435.4141
Epoch 833/5000
1/1 [==============

1/1 [==============================] - 0s 13ms/step - loss: 31022.3672 - val_loss: 78487.3906
Epoch 900/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3652 - val_loss: 78483.9688
Epoch 901/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3594 - val_loss: 78480.5469
Epoch 902/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3535 - val_loss: 78477.2656
Epoch 903/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3516 - val_loss: 78473.9844
Epoch 904/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3516 - val_loss: 78470.8359
Epoch 905/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3496 - val_loss: 78467.6875
Epoch 906/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3438 - val_loss: 78464.6797
Epoch 907/5000
1/1 [==============================] - 0s 13ms/step - loss: 31022.3379 - val_loss: 78461.6719
Epoch 908/5000
1/1 [==============

1/1 [==============================] - 0s 12ms/step - loss: 31022.7793 - val_loss: 78707.9219
Epoch 975/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.7578 - val_loss: 78700.1094
Epoch 976/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.7402 - val_loss: 78692.4375
Epoch 977/5000
1/1 [==============================] - 0s 12ms/step - loss: 22074.3223 - val_loss: 83702.8516
Epoch 978/5000
1/1 [==============================] - 0s 12ms/step - loss: 31093.9375 - val_loss: 81950.3828
Epoch 979/5000
1/1 [==============================] - 0s 12ms/step - loss: 31052.2500 - val_loss: 80834.4297
Epoch 980/5000
1/1 [==============================] - 0s 16ms/step - loss: 31035.1504 - val_loss: 80120.0938
Epoch 981/5000
1/1 [==============================] - 0s 12ms/step - loss: 31028.1230 - val_loss: 79660.7891
Epoch 982/5000
1/1 [==============================] - 0s 12ms/step - loss: 31025.2207 - val_loss: 79363.5312
Epoch 983/5000
1/1 [==============

Epoch 1049/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.4082 - val_loss: 78519.6719
Epoch 1050/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.4004 - val_loss: 78515.5703
Epoch 1051/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3965 - val_loss: 78511.6016
Epoch 1052/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3809 - val_loss: 78507.6328
Epoch 1053/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3887 - val_loss: 78503.8047
Epoch 1054/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3809 - val_loss: 78500.1094
Epoch 1055/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3770 - val_loss: 78496.4141
Epoch 1056/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3730 - val_loss: 78492.8594
Epoch 1057/5000
1/1 [==============================] - 0s 12ms/step - loss: 31022.3691 - val_loss: 78489.3047
Epoch 1058

In [47]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 33ms/step
[4089.333](test_target) - [[3256.854]](prediction) = 832.4789960937501


In [48]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [49]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Santa Catarina - IDH,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Santa Catarina - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Longevidade,Santa Catarina - IDH Educacao
0,-1.413268,-1.263964,1.142102,1.390320,0.721775,1.330126,1.165029,1.396038,1.335266,-1.330895,...,-1.413983,0.338939,-1.176663,-0.172911,1.408109,1.241766,1.097062,-1.273645,-1.396499,-1.399913
1,0.751418,0.082621,0.151239,-0.470978,-1.414111,-0.249041,0.111769,-0.502296,-0.264151,0.251276,...,0.684859,-1.358520,-0.091078,-1.129101,-0.817727,-0.034784,0.224348,0.104495,0.891494,0.526225
2,0.661850,1.181342,-1.293341,-0.919342,0.692336,-1.081084,-1.276798,-0.893743,-1.071116,1.079619,...,0.729124,1.019581,1.267741,1.302011,-0.590382,-1.206982,-1.321410,1.169149,0.505005,0.873688


train_target:


,Santa Catarina - Consumo de Cimento (t)
0,3017.084229
1,2776.217486
2,2891.489000


test_input:


...

,Santa Catarina - IDH,Santa Catarina - PIB - Estadual,Santa Catarina - PIB - Construção Civil,Santa Catarina - PIB - Per Capita,Santa Catarina - PIB - Preços de Mercado,Santa Catarina - Produção de Cimento (t),Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Santa Catarina - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Santa Catarina - IDH Longevidade,Santa Catarina - IDH Educacao
6,-1.851289,-1.212767,-0.018643,-1.576849,-1.943166,2.016188,1.16956,2.035192,0.985175,-0.705937,...,-1.368185,-1.412171,-0.143901,0.697496,-1.859643,-1.59586,-0.998008,0.317671,-1.647513,0.266084


test_target:


,Santa Catarina - Consumo de Cimento (t)
6,4089.333


Epoch 1118: early stopping
1/1 [==============================] - 0s 34ms/step
Error: 832.4789960937501




[2889.695556640625, 2933.98291015625, 3056.685302734375, 3256.85400390625]

In [50]:
display(targets)
display(predictions)

[2976.485, 3258.027, 3537.032, 4089.333]

[2889.695556640625, 2933.98291015625, 3056.685302734375, 3256.85400390625]

In [51]:
mae = mean_absolute_error(predictions, targets)
mae

430.9148066406251

In [52]:
porcentage = mae/np.mean(targets)
porcentage

0.12435426896598968